<a href="https://colab.research.google.com/github/JoseFPortoles/OpenCV_Examples/blob/master/TMF_Making_people_invisible.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Invisible walker
Here a temporal median filter was used to process a [video](https://youtu.be/j7tH1N01q3w) of a walking person, making the person invisible while still showing the background. The direction of the walker's shadow combined with the particular choice of frames involved in computing the median produce the effect of a ghostly shadow walking on its own. 

The code below was run in Google Colab. It can be reused by filling the `input_path` to point to an input video and `output_path` to name the resulting output video.

Some play with the median calculation parameters might be needed to achieve a particular result; essentially the speed of movements of different objects. Some parameter choices for the above linked video, for instance, would make the walker invisible but still display his feet, as the feet pixels remain for a longer time in the same position than the rest of the body.

Also, the sound track from the original video was merged afterwards using VLC, as the code below only processes the video stream.



In [22]:
import cv2 as cv
import numpy as np
import matplotlib.pyplot as plt

from google.colab.patches import cv2_imshow

def align_TM(frames, method='cv.TM_CCOEFF', subframe_loc = (0,-1,0,-1)):
    ''' Aligns an array of grayscale frames using template matching. 

        Arguments:
            frames: numpy array containing the grayscale frameset 

            method: string specifying the match template method
    
                Some standard match template method names:
                    'cv.TM_CCOEFF',
                    'cv.TM_CCOEFF_NORMED',
                    'cv.TM_CCORR',
                    'cv.TM_CCORR_NORMED',
                    'cv.TM_SQDIFF',
                    'cv.TM_SQDIFF_NORMED'

            subframe_loc: tuple specifyng subframe area location
                (subframe_top,
                subframe_left,
                subframe_right,
                subframe_bottom)

                The subframe area of each frame will be aligned to the first frame
                in the frameset.
    '''
    aligned = np.zeros(frames.shape, dtype="uint8")
    aligned[0] = frames[0]
    height, width = frames.shape[1:]

    meth = eval(method)

    subf_top = subframe_loc[0]
    subf_bottom = subframe_loc[1]
    subf_left = subframe_loc[2]
    subf_right = subframe_loc[3]

    subf_w = subf_right - subf_left
    subf_h = subf_bottom - subf_top

    for i, frame in enumerate(frames[1:]):
        subframe = frame[
                         subf_top : subf_bottom,
                         subf_left : subf_right 
                         ]
        mt = cv.matchTemplate(subframe, frames[0], meth)
        min_val, max_val, min_loc, max_loc = cv.minMaxLoc(mt)

        # If the method is TM_SQDIFF or TM_SQDIFF_NORMED, take minimum

        if meth in [cv.TM_SQDIFF, cv.TM_SQDIFF_NORMED]:
            top_left = min_loc
        else:
            top_left = max_loc

        # Affine translation

        M = np.float32([
                        [1,0,top_left[0] - width + subf_w],
                        [0,1,top_left[1] - height + subf_h]
                        ]) 
        aligned_frame = cv.warpAffine(
            frame,
            M,
            (width,height))
        aligned[i+1] = aligned_frame.astype("uint8")
    return aligned

# Start time in msec.

imsec = 0 

# Median calculation parameters

nframes = 15 # Nr of frames used to calculate the median
framestep = 5 # Nr of frames between consecutive frames
subframe_loc = (-200, -1, -200, -1)

# Template-matching method

methods = ['cv.TM_CCOEFF',
           'cv.TM_CCOEFF_NORMED',
           'cv.TM_CCORR',
           'cv.TM_CCORR_NORMED',
           'cv.TM_SQDIFF',
           'cv.TM_SQDIFF_NORMED'
           ]
method = methods[0]

# Input/Output paths

input_path = '/content/drive/My Drive/VID_20200920_093134.mp4'
output_path = '/content/drive/My Drive/output.avi'


# Create videocapture from file

cap = cv.VideoCapture(input_path)

# Get frame geometry and framerate

width = int(
    cap.get(cv.CAP_PROP_FRAME_WIDTH))
height = int(
    cap.get(cv.CAP_PROP_FRAME_HEIGHT))
fps = cap.get(cv.CAP_PROP_FPS)
img = np.zeros((height, width))

# Define the codec and create VideoWriter object

fourcc = cv.VideoWriter_fourcc(*"XVID")
out = cv.VideoWriter(output_path, fourcc, fps, (width,  height))

# Set the starting time and frame

cap.set(cv.CAP_PROP_POS_MSEC, imsec)
iframe = int(cap.get(cv.CAP_PROP_POS_FRAMES))

# Main loop goes frame by frame
while True:
    # Indexes for median computation starting on current frame (iframe)
    frameIdxs = np.array(
        [x for x in range(iframe,
                          iframe + framestep*nframes,
                          framestep) 
        ])
    print('frame(' + str(frameIdxs[0]) +'): ' + str(frameIdxs))
  
    frames = []

    for fid in frameIdxs:
        cap.set(cv.CAP_PROP_POS_FRAMES, fid)
        ret, frame = cap.read()
        if ret is not True:
            print('End of source video reached. Exiting...')
            break

        gray = cv.cvtColor(frame, cv.COLOR_BGR2GRAY)
        frames.append(gray)
  
    if ret is not True:
        break

    frames = np.array(frames)
    frames = align_TM(frames, method = method, subframe_loc = subframe_loc)
    medframe = np.median(frames, axis=0).astype("uint8")
    medframe = cv.cvtColor(medframe,cv.COLOR_GRAY2RGB)

    if out.isOpened() is not True:
        print("Output Writer is not open")
        break

    out.write(medframe)
    iframe += 1
  
print('Releasing videos. iframe = ' + str(iframe))
print('Frame count: ' + str(cap.get(cv.CAP_PROP_FRAME_COUNT)))
print('FPS: ' + str(cap.get(cv.CAP_PROP_FPS)))
cap.release()
out.release()

frame(0): [ 0  5 10 15 20 25 30 35 40 45 50 55 60 65 70]
frame(1): [ 1  6 11 16 21 26 31 36 41 46 51 56 61 66 71]
frame(2): [ 2  7 12 17 22 27 32 37 42 47 52 57 62 67 72]
frame(3): [ 3  8 13 18 23 28 33 38 43 48 53 58 63 68 73]
frame(4): [ 4  9 14 19 24 29 34 39 44 49 54 59 64 69 74]
frame(5): [ 5 10 15 20 25 30 35 40 45 50 55 60 65 70 75]
frame(6): [ 6 11 16 21 26 31 36 41 46 51 56 61 66 71 76]
frame(7): [ 7 12 17 22 27 32 37 42 47 52 57 62 67 72 77]
frame(8): [ 8 13 18 23 28 33 38 43 48 53 58 63 68 73 78]
frame(9): [ 9 14 19 24 29 34 39 44 49 54 59 64 69 74 79]
frame(10): [10 15 20 25 30 35 40 45 50 55 60 65 70 75 80]
frame(11): [11 16 21 26 31 36 41 46 51 56 61 66 71 76 81]
frame(12): [12 17 22 27 32 37 42 47 52 57 62 67 72 77 82]
frame(13): [13 18 23 28 33 38 43 48 53 58 63 68 73 78 83]
frame(14): [14 19 24 29 34 39 44 49 54 59 64 69 74 79 84]
frame(15): [15 20 25 30 35 40 45 50 55 60 65 70 75 80 85]
frame(16): [16 21 26 31 36 41 46 51 56 61 66 71 76 81 86]
frame(17): [17 22 27 32 